1 imagenet256のラベル付けを変更する 

2 imagenet_class_index_lower.jsonにimagenet-256のデータセットのラベルと会うように文字列を修正（手動＋コード）

3 事前学習済みのresnet50を用いてimagenet-256のデータセットの画像分類

In [22]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

# 1. 前処理
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# 2. データセット
data_dir = "./imagenet-256"
val_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# 3. モデル
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model = model.to(device)
model.eval()

# 4. ImageNet クラスインデックスを読み込み
with open("imagenet_class_index_lower.json", "r") as f:
    class_idx = json.load(f)

# name → index の辞書を作成（空白はアンダースコアに変換）
name_to_idx = {}
for k, v in class_idx.items():
    idx = int(k)
    synset, name = v
    # "afghan hound" → "afghan_hound"
    norm_name = name.replace(" ", "_")
    name_to_idx[norm_name] = idx

# abc順 (val_dataset.classes) → ImageNetインデックス
abc2idx = {}
missing = []
for cls_name in val_dataset.classes:
    if cls_name in name_to_idx:
        abc2idx[cls_name] = name_to_idx[cls_name]
    else:
        missing.append(cls_name)

print(f"[INFO] マッチできたクラス数: {len(abc2idx)}/{len(val_dataset.classes)}")
if missing:
    print(f"[WARN] マッチできなかったクラス例: {missing[:10]} ...")

# 5. 精度計算
correct1, correct5, total = 0, 0, 0

# ...existing code...

correct_top1_paths = []
correct_top5_paths = []

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(tqdm(val_loader, desc="Evaluating", unit="batch")):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        valid_indices = []
        valid_labels_idx = []
        valid_paths = []
        for i, l in enumerate(labels.cpu()):
            cls_name = val_dataset.classes[l]
            if cls_name in abc2idx:
                valid_indices.append(i)
                valid_labels_idx.append(abc2idx[cls_name])
                img_idx = batch_idx * val_loader.batch_size + i
                img_path, _ = val_dataset.samples[img_idx]
                valid_paths.append(img_path)

        if not valid_indices:
            continue

        valid_images = images[valid_indices]
        valid_labels_idx = torch.tensor(valid_labels_idx).to(device)
        valid_outputs = outputs[valid_indices]

        # top-1
        _, pred1 = valid_outputs.max(1)
        correct_mask1 = pred1.eq(valid_labels_idx)
        for idx, is_correct in enumerate(correct_mask1.cpu()):
            if is_correct:
                correct_top1_paths.append(valid_paths[idx])

        # top-5
        _, pred5 = valid_outputs.topk(5, 1, True, True)
        correct_mask5 = pred5.eq(valid_labels_idx.view(-1,1)).any(dim=1)
        for idx, is_correct in enumerate(correct_mask5.cpu()):
            if is_correct:
                correct_top5_paths.append(valid_paths[idx])

        correct1 += correct_mask1.sum().item()
        correct5 += correct_mask5.sum().item()
        total += len(valid_indices)

print(f"Top-1 Accuracy: {100 * correct1 / total:.2f}%")
print(f"Top-5 Accuracy: {100 * correct5 / total:.2f}%")

# 正解画像パスをファイルに保存
with open("correct_top1_images.txt", "w") as f:
    for path in correct_top1_paths:
        f.write(path + "\n")

with open("correct_top5_images.txt", "w") as f:
    for path in correct_top5_paths:
        f.write(path + "\n")

print(f"Top-1で正解した画像数: {len(correct_top1_paths)}")
print(f"Top-5で正解した画像数: {len(correct_top5_paths)}")
# ...existing code...


[INFO] マッチできたクラス数: 999/1000
[WARN] マッチできなかったクラス例: ['tank_suit'] ...


Evaluating: 100%|██████████| 4218/4218 [12:06<00:00,  5.81batch/s]


Top-1 Accuracy: 85.51%
Top-5 Accuracy: 97.17%
Top-1で正解した画像数: 461169
Top-5で正解した画像数: 524073


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
from tqdm import tqdm
import json
import os

# 変換は元と同じ
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# モデル
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model = model.to(device)
model.eval()

# クラス名→index辞書
with open("imagenet_class_index_lower.json", "r") as f:
    class_idx = json.load(f)
name_to_idx = {}
for k, v in class_idx.items():
    idx = int(k)
    synset, name = v
    norm_name = name.replace(" ", "_")
    name_to_idx[norm_name] = idx

# 画像パスリストを読み込む
with open("correct_top1_images.txt") as f:
    image_paths = [line.strip() for line in f]

# パスからクラス名を取得する関数
def get_class_from_path(path):
    # 例: ./imagenet-256/abacus/xxx.jpg → abacus
    return os.path.basename(os.path.dirname(path))

# 検証
correct_top1 = 0
correct_top5 = 0
for path in tqdm(image_paths, desc="Re-evaluating", unit="img"):
    img = Image.open(path).convert("RGB")
    input_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_tensor)
        pred1 = output.argmax(1).item()
        _, pred5 = output.topk(5, 1, True, True)
        pred5 = pred5.cpu().numpy()[0]
    cls_name = get_class_from_path(path)
    if cls_name in name_to_idx:
        label_idx = name_to_idx[cls_name]
        if pred1 == label_idx:
            correct_top1 += 1
        if label_idx in pred5:
            correct_top5 += 1

print(f"再分類でTop-1正解した画像数: {correct_top1}/{len(image_paths)}")
print(f"再分類でTop-5正解した画像

KeyboardInterrupt: 